In [1]:
import glob
import os
from collections import Counter
from itertools import combinations

import numpy as np
np.random.seed(18012023)

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import seaborn as sb

from sklearn.metrics import pairwise_distances
import lxml.etree
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = 'Arial'

In this notebook, we run the intertext retrieval (with the calibrated parameters) on all of the narrative (epic) texts available on the *Cd-rom Middelnederlands*. We dump the results to a spreadsheet that we will statistically analyze in the next notebook.

First we make sure that the output direcotry exists:

In [2]:
fig_dir = '../figures'
if not os.path.isdir(fig_dir):
    os.mkdir(fig_dir)

We load the metadata (with the subgenre information etc.):

In [3]:
meta_df = pd.read_excel('../data/metadata_corrected.xlsx')
meta_df = meta_df[meta_df['exclude'] != 'x']
meta_df.sample(10)

,id,title,author,provenance,date_range,genre,subgenre,exclude
201,glose_op_het_sint-jansevangelie,Sinte Jans ewangelium alsoe 't Augustijnken gheexponeert heeft,Augustijnken,cdrom-mnl,1400-1420,Epiek,Didactiek,NaN
64,florigout_fragm_db,Florigout,NaN,cdrom-mnl,1375-1400,Epiek,Ridder,NaN
112,loyhier_en_malaert_fragm_g,Loyhier en Malaert,NaN,cdrom-mnl,1350-1400,Epiek,Karel,NaN
87,historie_van_troyen,Historie van Troyen,Jacob van Maerlant,cdrom-mnl,1465-1485,Epiek,Ridder,NaN
169,renout_van_montalbaen_fragm_s,Renout van Montalbaen,NaN,cdrom-mnl,1490-1510,Epiek,Karel,NaN
215,spiegel_historiael__1_3_4_maerlant,Spiegel historiael (P1-P4),Jacob van Maerlant,cdrom-mnl,1300-1325,Epiek,Historiografie,NaN
211,spiegel_der_sonden,Spiegel der sonden,NaN,cdrom-mnl,1440-1460,Epiek,Didactiek,NaN
124,madelgijs_fragm_r,Madelgijs,NaN,cdrom-mnl,1340-1360,Epiek,Karel,NaN
167,renout_van_montalbaen_fragm_be,Renout van Montalbaen,NaN,cdrom-mnl,1340-1360,Epiek,Karel,NaN
240,theophilus,Theophilus,NaN,cdrom-mnl,1400-1420,Epiek,Heiligenleven,NaN


Next, we redefine the extraction functions from the previous notebooks that we'll need again:

In [4]:
def get_verse_groups(verses, size=2, intertexts=False):
    for i in range(len(verses) - (size - 1)):
        if not intertexts:
            yield ' / '.join(verses[i : i + size])
        else:
            its = Counter(verses[i : i + size])
            if None in its:
                yield None
            elif len(its) > 1:
                yield 'overlap'
            else:
                yield list(its.keys())[0]

In [5]:
def parse_xml(fn, rm_interpol=False):
    try:
        tree = lxml.etree.parse(fn)
    except OSError:
        print(f'- Could not load {fn}')
        return None
        
    if rm_interpol:
        for interpolation in tree.xpath("//interpolation"):
            interpolation.getparent().remove(interpolation)
        
    for line_node in tree.iterfind('.//l'):
        try:
            intertext_id = line_node.attrib['intertext']
        except KeyError:
            intertext_id = None
        
        tokens_ = line_node.attrib['tokens'].split()
        lemmas_ = []
        
        lemma_tags = [l.text for l in line_node.iterfind('.//lemma')]
        pos_tags = [p.text for p in line_node.iterfind('.//pos')]
        
        for lemma, pos in zip(lemma_tags, pos_tags):
            for l, p in zip(lemma.split('+'), pos.split('+')):
                    if p == 'n(prop)':
                        lemmas_.append('n(prop)')
                    else:
                        lemmas_.append(l)
    
        yield tokens_, lemmas_, intertext_id

In [6]:
GROUP_SIZE = 2

titles, tokens, lemmas, intertexts = [], [], [], []

for title, group in tqdm(meta_df.groupby('title')):
    work_tokens, work_lemmas, work_intertexts = [], [], []
    
    for id_ in sorted(group['id']):
        for tok, lem, intertext_id in parse_xml(f'../data/xml/{id_}.xml', rm_interpol=True):
            work_tokens.append(tok)
            work_lemmas.append(lem)
            work_intertexts.append(intertext_id)
    
    verse_tokens = [' '.join(v) for v in work_tokens]
    verse_lemmas = [' '.join(v) for v in work_lemmas]

    verse_group_tokens = list(get_verse_groups(verse_tokens, size=GROUP_SIZE))
    verse_group_lemmas = list(get_verse_groups(verse_lemmas, size=GROUP_SIZE))
    verse_group_intertexts = list(get_verse_groups(work_intertexts, size=GROUP_SIZE, intertexts=True))

    tokens.extend(verse_group_tokens)
    lemmas.extend(verse_group_lemmas)
    intertexts.extend(verse_group_intertexts)
    titles.extend([title] * len(verse_group_lemmas))

100%|██████████| 204/204 [00:15<00:00, 12.95it/s]


Now, we can load all the verse groups from all the texts, as well as the associated metdata into a single dataframe:

In [7]:
df = pd.DataFrame(zip(titles, tokens, lemmas, intertexts), columns=('title', 'tokens', 'lemmas', 'intertext'))

Finally, we can add the separate column for the lemmas in rhyming position:

In [8]:
def tokenizer(text):
    return text.replace(' / ', ' ').lower().strip().split()

def add_rhyme_column(df):
    rhyme_words = []
    for lemmas in df['lemmas']:
        rhymes = []
        for verse in lemmas.split(' / '):
            rhymes.append(verse.strip().split()[-1])
        rhyme_words.append(' '.join(rhymes))
    df['rhyme'] = rhyme_words
    return df

In [9]:
df = add_rhyme_column(df)
df.head()

,title,tokens,lemmas,intertext,rhyme
0,AB recht ende averecht,Aensiet dese vrouwen hoe si gaen / Besiet hoe haer tuten staen,aanzien deze vrouw hoe zij gaan / bezien hoe zij de staan,None,gaan staan
1,AB recht ende averecht,Besiet hoe haer tuten staen / Claer dat si hen blanketten,bezien hoe zij de staan / klaar dat zij zij n(prop),None,staan n(prop)
2,AB recht ende averecht,Claer dat si hen blanketten / Die cleeder soe lanc dat si hen letten,klaar dat zij zij n(prop) / de kleed zo lang dat zij zij letten,None,n(prop) letten
3,AB recht ende averecht,Die cleeder soe lanc dat si hen letten / Ende sleypen hen nae al op die eerde,de kleed zo lang dat zij zij letten / en slapen zij na al op de aarde,None,letten aarde
4,AB recht ende averecht,Ende sleypen hen nae al op die eerde / Fi diere vuylder hoverde,en slapen zij na al op de aarde / fi duur vouwer hovaardij,None,aarde hovaardij


In the previous notebook, these turned out to be the optimal hyperparameters for the retrieval system:

In [10]:
optim_vs = 9000
optim_rw = 0.1693877551020408
optim_th = 0.4217159759743446

In [11]:
def batch(iterable, n=1):
    l = iterable.shape[0]
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def pairwise(AX, BX, batch_size=5000):
    all_distances = None

    for ax in batch(AX, batch_size):
        if all_distances is None:
            all_distances = pairwise_distances(ax, BX, metric='cosine')
        else:
            all_distances = np.append(all_distances,
                                      pairwise_distances(ax, BX, metric='cosine'),
                                      axis=0)
    
    return all_distances

In [12]:
def retrieve_intertexts(title1, title2, distances, threshold):
    A = df[df['title'] == title1]
    B = df[df['title'] == title2]
    
    intertexts = []
    for idx1, idx2 in np.transpose(np.nonzero(distances < threshold)):
        tokens1 = A.iloc[idx1]['tokens']
        tokens2 = B.iloc[idx2]['tokens']
        distance = distances[idx1, idx2]
        intertexts.append((title1, title2, tokens1, tokens2, distance))
        
    return pd.DataFrame(intertexts, columns=['title1', 'title2', 'tokens1', 'tokens2', 'distance'])

def plot_min_distance_nn(t1, t2, min_dists, threshold, rolling_size=1000):
    mu = min_dists.rolling(rolling_size).mean()
    fig, ax = plt.subplots()
    mu.plot(ax=ax)
    ax.axhline(threshold, ls='--', c='grey')
    ax.set_ylim(0, 1)
    ax.set_xlabel(t1)
    ax.set_ylabel('Distance to NN in ' + t2)
    return ax

We well restrict the analysis to all narrative texts (in rhyming couplets), or the so-called "Epiek":

In [13]:
chiv_titles = meta_df[meta_df['genre'] == 'Epiek']['title'].unique()

In [14]:
vec = TfidfVectorizer(max_features=optim_vs, min_df=2,
                      tokenizer=tokenizer, token_pattern=None).fit(df['lemmas'])

We will exhaustively combine all texts, but we make sure to only combine two texts once (because our retrieval is non-directional):

In [15]:
combs = sorted([sorted(comb) for comb in combinations(chiv_titles, 2)])
combs

[['Alexanders geesten', 'Alexiuslegende'],
 ['Alexanders geesten', 'Anatomie van de mens'],
 ['Alexanders geesten', 'Arturs doet'],
 ['Alexanders geesten', 'Aubri de Borgengoen'],
 ['Alexanders geesten', 'Barlaam en Josaphat'],
 ['Alexanders geesten', 'Beatrijs'],
 ['Alexanders geesten', 'Bediedenisse van der missen'],
 ['Alexanders geesten', 'Beerte metten breden voeten'],
 ['Alexanders geesten', 'Beginsel der oorlogen van Luyck'],
 ['Alexanders geesten', 'Boec Exemplaer'],
 ['Alexanders geesten', 'Boec van den houte'],
 ['Alexanders geesten', 'Boec vander wraken'],
 ['Alexanders geesten', 'Boeve van Hamtone'],
 ['Alexanders geesten', 'Borchgrave van Couchi'],
 ['Alexanders geesten', 'Borchgravinne van Vergi (1)'],
 ['Alexanders geesten', 'Borchgravinne van Vergi (2)'],
 ['Alexanders geesten', 'Boudewijn van Seborch'],
 ['Alexanders geesten', 'Brabantsche yeesten (B1-5)'],
 ['Alexanders geesten', 'Brabantsche yeesten (B6)'],
 ['Alexanders geesten', 'Brabantsche yeesten (B7)'],
 ['Alex

In [16]:
lookup = {}
for t1, t2 in combs:
    try:
        lookup[t1].append(t2)
    except KeyError:
        lookup[t1] = [t2]
lookup

{'Alexanders geesten': ['Alexiuslegende',
  'Anatomie van de mens',
  'Arturs doet',
  'Aubri de Borgengoen',
  'Barlaam en Josaphat',
  'Beatrijs',
  'Bediedenisse van der missen',
  'Beerte metten breden voeten',
  'Beginsel der oorlogen van Luyck',
  'Boec Exemplaer',
  'Boec van den houte',
  'Boec vander wraken',
  'Boeve van Hamtone',
  'Borchgrave van Couchi',
  'Borchgravinne van Vergi (1)',
  'Borchgravinne van Vergi (2)',
  'Boudewijn van Seborch',
  'Brabantsche yeesten (B1-5)',
  'Brabantsche yeesten (B6)',
  'Brabantsche yeesten (B7)',
  "Chanson d'Aspremont",
  'Collectief lunarium I',
  'Collectief lunarium II',
  'Collectief lunarium III',
  'Commentaar op het Hooglied',
  'Cyromanchie van den pape van den Hamme',
  'Der leken spieghel',
  'Der mannen ende vrouwen heimelijcheit',
  'Der minnen loep',
  'Der naturen bloeme',
  'Der vrouwen heimelijcheit',
  'Der ystorien bloeme',
  'Dietsche Lucidarius',
  'Dietsche doctrinale',
  'Enaamse Codex',
  'Esopet',
  'Expositi

The following block (which takes a while to run) runs over all text combinations and calculates the distance between all of the respective verse groups. We keep track of the length of both texts in the pair and count the number of "hits", or combinations of verse groups that have a smaller distance than the threshold value, which we found to be optimal in the manually annotated data:

In [17]:
results, intertexts = [], []

for cnt, t1 in enumerate(lookup):
    print(t1, f'{cnt+1}/{len(lookup)}')
    a1 = meta_df[meta_df['title'] == t1]['author'].iloc[0]
    g1 = meta_df[meta_df['title'] == t1]['genre'].iloc[0]
    sg1 = meta_df[meta_df['title'] == t1]['subgenre'].iloc[0]
    
    A = df[df['title'] == t1]
    AX = vec.transform(A['lemmas']) + optim_rw * vec.transform(A['rhyme'])
    
    for t2 in tqdm(lookup[t1]):
        a2 = meta_df[meta_df['title'] == t2]['author'].iloc[0] 
        g2 = meta_df[meta_df['title'] == t2]['genre'].iloc[0]
        sg2 = meta_df[meta_df['title'] == t2]['subgenre'].iloc[0]
        
        B = df[df['title'] == t2]
        BX = vec.transform(B['lemmas']) + optim_rw * vec.transform(B['rhyme'])
        
        # get NNs for statistics
        distances = pairwise(AX, BX, batch_size=10000)
        l1, l2 = len(df[df['title'] == t1]), len(df[df['title'] == t2])
        hits = np.sum(distances <= optim_th)
        results.append((t1, t2, l1, l2, hits, g1, g2, sg1, sg2, a1, a2))
    
        # collect actual intertexts
        #intertexts.append(retrieve_intertexts(t1, t2, distances=distances, threshold=threshold))
    
#intertexts = pd.concat(intertexts)
#intertexts.to_excel('../figures/intertexts.xlsx', index=False)

results = pd.DataFrame(results, columns=['t1', 't2',
                                         'l1', 'l2', 'hits',
                                         'genre1', 'genre2',
                                         'subgenre1', 'subgenre2',
                                         'author1', 'author2'])
results['hit_ratio'] = results['hits'] / (results['l1'] * results['l2'])
results = results.sort_values(by='hit_ratio', ascending=False)
results.to_excel('../figures/statistics.xlsx', index=False)
results

Alexanders geesten 1/153


100%|██████████| 153/153 [02:35<00:00,  1.02s/it]


Alexiuslegende 2/153


100%|██████████| 152/152 [00:16<00:00,  9.40it/s]


Anatomie van de mens 3/153


...

Willem van Oringen 152/153


100%|██████████| 2/2 [00:00<00:00, 10.54it/s]


Wisselau 153/153


100%|██████████| 1/1 [00:00<00:00, 10.41it/s]


,t1,t2,l1,l2,hits,genre1,genre2,subgenre1,subgenre2,author1,author2,hit_ratio
9352,Noch meer van wiven,Van der wive wonderlijcheit (lange versie),111,225,149,Epiek,Epiek,Didactiek,Didactiek,NaN,NaN,0.005966
6767,Gwidekijn van Sassen,Ongeïdentificeerd (4),199,139,35,Epiek,Epiek,Karel,Karel,NaN,NaN,0.001265
9784,Ongeïdentificeerd (4),Roman van Cassant,139,29,5,Epiek,Epiek,Karel,Kruisvaart,NaN,NaN,0.001240
5296,Fierabras,Roman van Cassant,314,29,8,Epiek,Epiek,Karel,Kruisvaart,NaN,NaN,0.000879
6787,Gwidekijn van Sassen,Roman van Cassant,199,29,5,Epiek,Epiek,Karel,Kruisvaart,NaN,NaN,0.000866
...,...,...,...,...,...,...,...,...,...,...,...,...
9657,Ongeïdentificeerd (2),Roman van Cassant,2,29,0,Epiek,Epiek,Karel,Kruisvaart,NaN,NaN,0.000000
9658,Ongeïdentificeerd (2),Rose,2,14409,0,Epiek,Epiek,Karel,Didactiek,NaN,NaN,0.000000
9659,Ongeïdentificeerd (2),Rubben,2,243,0,Epiek,Epiek,Karel,Didactiek,NaN,NaN,0.000000
9660,Ongeïdentificeerd (2),Saladin,2,143,0,Epiek,Epiek,Karel,Kruisvaart,NaN,NaN,0.000000
